# I) Overview  
- This will concantenate .fastq.gz files for samples that were run in multiple lanes
- This will work for both single end (contain R1 only) or paired-end (contain R1 and R2)
- Will rename the file using the figueroa lab ID e.g 'EA00001'
- Requires a tab seperated SampleID.txt file with headers. The first column is the 'core_id' (id that core used to name the files, typically just a number) and the second column is 'figueroa_id', the name that the files will be renamed as
- This assumes that:
    - files end with R1_001.fastq.gz or R2_001.fastq.gz
    - All .fastq.gz files are in the same directory




In [ ]:
import pandas as pd

In [ ]:
input_dir = "/raw_data/"
output_dir = "/ChIPseq/fastq/"
sampleID = pd.read_csv('/SampleID.txt', sep='\t')
sample_hash_id = dict(zip(sampleID.core_id, sampleID.figueroa_id))

In [ ]:
files_R1= !find {input_dir} | grep _R1_001.fastq.gz | grep -v "Undetermined"
files_R2= !find {input_dir} | grep _R1_002.fastq.gz | grep -v "Undetermined"

In [ ]:
files_R1_df=pd.DataFrame(files_R1)
files_R2_df=pd.DataFrame(files_R2)

In [ ]:
def cat_fastq(df):
    if not df.empty:
        df['fastq_id']= df[0].apply(lambda x: x.split("/")[-1].split("_")[1])
        df_g = df.groupby('fastq_id')
        final_cmd = []
        start =1
        for k, v in df_g:
            files_to_concat = ' '.join(v[0].values.tolist())
            new_name = output_dir + sample_hash_id[k]
            files_to_concat_just_name = ' '.join([f.split('/')[-1] for f  in files_to_concat.split(' ') ])
            cmd = f' (cd {input_dir} ; cat {files_to_concat_just_name} > {new_name}.fastq.gz)'
            final_cmd.append(cmd)
        cmd_torun=' & '.join(final_cmd)
        #!$cmd_torun
        print(cmd_torun)
    else:
        print('DataFrame is empty!')


In [ ]:
cat_fastq(files_R2_df)

In [ ]:
cat_fastq(files_R1_df)